In [ ]:
import pandas as pd

# load the data
file_path = "credit_risk_dataset.csv"
data = pd.read_csv(file_path)

# display the first few rows of the dataset
print("First few rows of the dataset:")
print(data.head())

# check for missing values
print("\nMissing values in the dataset:")
print(data.isnull().sum())

# display the data types of each column
print("\nData types of each column:")
print(data.dtypes)